In [1]:
import sys
sys.path.append('../../30_data_tools')

In [2]:
from helper import load_dotenv
from get_labelstudio_data import get_tasks, get_results_of_project
from file_interaction import open_img

In [3]:
import pandas as pd
import requests
from PIL import Image
from datetime import datetime
from tqdm.auto import tqdm
import sqlite3

In [4]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
config = load_dotenv()

In [6]:
con = sqlite3.connect(config['DB_PATH'])

In [7]:
tasks = get_tasks(2)
results = get_results_of_project(2)

In [8]:
masks = pd.read_sql(
    '''
        SELECT m.*, gi.timestamp FROM mask m
        LEFT JOIN generic_image gi
        ON
            m.pdf_filename=gi.pdf_filename AND
            m.job=gi.job AND
            m.type=gi.type AND
            m.variant_name=gi.variant_name AND
            m.method=gi.method AND
            m.idx=gi.idx
        WHERE ssim < 0.95
    ''',
    con
)

masks.loc[
    :,
    'img_name'
] = masks.apply(lambda val: f"{ val.job }.{ val.pdf_filename }.{ val.method }.{ val.idx }.4c_{ config['LOFI_DPI'] }.jpg", axis=1)

masks = masks.loc[
    masks.img_name.isin(set([r['img_name'] for r in results])) == False
]

masks = masks.loc[
    masks.mask_id.str.match('^temp_.+')
]

In [9]:
def create_annotation( task, masks, results ):
    with open_img( (task['data_path'],'azure') ) as img:
        img_size = img.size
     
    headers = {'Authorization': f'Token { config["LABEL_STUDIO_TOKEN"] }'}
    json_data = {
        "completed_by": 1,
        "result": [
            {
                "original_width": img_size[0],
                "original_height": img_size[1],
                "image_rotation": 0,
                "value": {
                    "x": m[0] / img_size[0] * 100,
                    "y": m[1] / img_size[1] * 100,
                    "width": m[2] / img_size[0] * 100,
                    "height": m[3] / img_size[1] * 100,
                    "rotation": 0,
                    "rectanglelabels": [
                        "unchecked_moire"
                    ]
                },
                "from_name": "label",
                "to_name": "image",
                "type": "rectanglelabels",
                "origin": "manual"
            } for m in masks
        ],
        "was_cancelled": False,
        "ground_truth": False,
        "draft_created_at": datetime.now().isoformat(),
        "task": task["id"],
        "project": task["project_id"],
        "updated_by": 1,
        "parent_prediction": None,
        "parent_annotation": None
    }

    if len([r for r in results if r['img_name'] == task['data_path'].split('/')[-1]]) == 0:    
        requests.post(
            f'http://localhost:8080/api/tasks/{ task["id"] }/annotations/',
            headers=headers,
            json=json_data
        )

In [10]:
for img_name in tqdm(masks.img_name.unique()):
    relevant_tasks = [t for t in tasks if img_name in t['data_path']]

    if len(relevant_tasks) > 0:
        task = relevant_tasks[0]
        
        masks_for_img = [
            [int(val) for val in m.split(';')]
            for m in
            masks.loc[
                masks.img_name == img_name
            ].bbox.values
        ]

        create_annotation( task, masks_for_img, results )

100%|██████████| 111/111 [00:53<00:00,  2.06it/s]
